Example copy:

1 Torchaudio load from S3:
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
response = client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
waveform, sample_rate = torchaudio.load(response['Body'])
plot_specgram(waveform, sample_rate, title="From S3")



Pre installatie
sudo apt install libsndfile1-dev
sudo apt install awscli

aws configure sso (Met informatie geleverd door Nick)

conda create --name humainr python=3.9
pip install boto3
pip install transformers"[speech, sentencepiece]"
pip install datasets


Verschillende imports voor het programma

In [ ]:
import boto3
import os
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration

Configuratie

In [91]:
import boto3
from datasets import load_dataset, Audio

boto3.setup_default_session(profile_name='AWSIntroTraining-408122842185')

s3 = boto3.resource('s3')
bucket = s3.Bucket('humainr-aws-intro-bucket')
#bucketname = 'humainr-aws-intro-bucket'

sync_path = 'data/new/'
arch_path = 'data/archive/'
test_file = 'data/new/common_voice_en_38497561.mp3'

for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)




obj = s3.Object(bucket_name='humainr-aws-intro-bucket', key='common_voice_en_38497561.mp3')
print(obj.bucket_name)
print(obj.key)
response = obj.get()
mydict = {"path": "common_voice_en_38497561.mp3","bytes":}
audio_bytes = response['Body'].read()


common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
humainr-aws-intro-bucket
common_voice_en_38497561.mp3


TypeError: decode_example() missing 1 required positional argument: 'self'

Verwerk de klaarstaande audiobestanden naar de AWS bucket

1 - Upload file naar S3
2 - Verplaats bestand van de sync_dir naar de arch_dir

In [65]:
import os, shutil

for entry in os.listdir(sync_path):
    if entry.endswith('.mp3'):
        bucket.upload_file(sync_path + entry, entry)
        shutil.move(sync_path + entry, arch_path + entry)




for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)
    #my_bucket_object.delete()

common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3


Example voor huggingface Speech2Text

In [78]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset, Audio
from miniaudio import SampleFormat, decode

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

# ds = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
### Laad de file/files in de dataset die transcribed moet worden
#ds = load_dataset("audiofolder", data_files=["data/new/common_voice_en_38497561.mp3"], split="train")

decoded_audio = decode(audio_bytes, nchannels=1, sample_rate=16000, output_format=SampleFormat.SIGNED32)

print(ds[0])

### Zorg ervoor dat de samplingrate 16000 is sinds het pretrained model dit verlangt
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

### Genereer 
inputs = processor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="pt")
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
transcription

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'audio': {'path': '/home/arnold/ML/humainr/data/new/common_voice_en_38497561.mp3', 'array': array([ 0.00000000e+00,  1.05738161e-11,  8.13509243e-12, ...,
       -8.14127998e-06,  5.09031233e-05, -1.78739399e-04]), 'sampling_rate': 32000}}


['they hastily drive to the plain and flee from the dangers of the mob']